See original StyleGAN2 ADA github repo here: https://github.com/NVlabs/stylegan2-ada-pytorch

By using a pre-trained model along with data heavy data augmentation, we can train our own GAN with a very limited dataset (<1000 images). 

In [ ]:
!pip install pillow
!pip install pyspng ninja imageio-ffmpeg==0.4.3
!git clone https://github.com/PDillis/stylegan3-fun
!pip install gdown
!pip install torch==1.11.0
%cd ./stylegan3-fun

In [ ]:
import os
from PIL import Image
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Read CSV files
train_csv = pd.read_csv("/kaggle/input/fetalhcdata/training_set/training_set_circumference.csv")
train_img = "/kaggle/input/fetalhcdata/training_set/training_set"
test_csv = pd.read_csv("/kaggle/input/fetalhcdata/test_set/test_set_circumference.csv")
test_img = "/kaggle/input/fetalhcdata/test_set/test_set"

def resize_images(image_ps, save_dir, size=(512, 512)):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    
    # Counter for image naming
    image_count = 0
    
    for image_path in image_ps:
        try:
            with Image.open(image_path) as img:
                img = img.resize(size, Image.LANCZOS)
                
                # Convert to RGB if image is in RGBA mode
                if img.mode == 'RGBA':
                    img = img.convert('RGB')
                
                # Create new filename
                new_file_name = f"image_{image_count}.jpg"
                output_path = os.path.join(save_dir, new_file_name)
                img.save(output_path)
                
                # Increment the image counter
                image_count += 1
        except Exception as e:
            print(f"Error processing {image_path}: {e}")

# Generate paths for the filtered data
train_paths = [os.path.join(train_img, filename) for filename in train_csv['filename']]
test_paths = [os.path.join(test_img, filename) for filename in test_csv['filename']]

# Combine the paths and resize images
output_directory = 'new_imgs'
resize_images(train_paths + test_paths, output_directory)

In [ ]:
!python dataset_tool.py --source=./new_imgs --dest=./datasets/FH.zip

In [ ]:
!python train.py --help

In [ ]:
!python train.py --outdir=./results --snap=5 --img-snap=5 --cfg=stylegan2 --data=./datasets/FH.zip --resume=afhqwild512 --augpipe=noise --gpus=2 --metrics=None --gamma=36 --batch=8

## Generate Synthetic Images
#### The code below is for RGB

In [ ]:
!pip install moviepy

In [ ]:
outdir = './results'
os.makedirs(outdir, exist_ok=True)
!python generate.py images --outdir=outdir --trunc=1 --seeds=0-9 --network=/kaggle/working/stylegan3-fun/results/00000-stylegan2-FH-gpus2-batch8-gamma8-resume_custom/network-snapshot-000100.pkl

In [ ]:
import cv2
import os

# Paths
input_dir = './results/00000-generate-images'
output_dir = './results/generated_grayscale'

# Create output directory if it does not exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Process each image
for filename in os.listdir(input_dir):
    # Build the path to the input file
    input_path = os.path.join(input_dir, filename)
    
    # Check if it's an image file
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
        # Read the image
        image = cv2.imread(input_path)
        
        # Convert to grayscale
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        # Build the path to the output file
        output_path = os.path.join(output_dir, filename)
        
        # Save the grayscale image
        cv2.imwrite(output_path, gray_image)

In [ ]:
!python dataset_tool.py --source=./results/generated_grayscale --dest=./results/gan_images.zip

#### The code below is for gray images

In [ ]:
import os
import numpy as np
import torch
import PIL.Image
import dnnlib
import legacy

def generate_images(network_pkl, seeds, truncation_psi, outdir):
    device = torch.device('cuda')
    
    # Load the pre-trained network
    with dnnlib.util.open_url(network_pkl) as f:
        G = legacy.load_network_pkl(f)['G_ema'].to(device)  # type: ignore

    os.makedirs(outdir, exist_ok=True)

    for seed in seeds:
        # Generate latent vectors
        z = torch.from_numpy(np.random.RandomState(seed).randn(1, G.z_dim)).to(device)
        img = G(z, None, truncation_psi=truncation_psi, noise_mode='const')

        # Convert the generated image to a format that can be saved
        img = (img * 127.5 + 128).clamp(0, 255).to(torch.uint8)
        img = img.permute(0, 2, 3, 1)  # [1, h, w, c]
        
        # Convert to numpy and save as grayscale image
        img_np = img[0].cpu().numpy().squeeze()  # [h, w]
        PIL.Image.fromarray(img_np, 'L').save(f'{outdir}/seed{seed:04d}.png')

# Example usage
network_pkl = '/kaggle/working/stylegan2-ada-pytorch/results/00000-fetalH-paper512-blit-resumecustom/network-snapshot-000080.pkl'
seeds = range(0, 1000)
truncation_psi = 1
outdir = './results/generated'

generate_images(network_pkl, seeds, truncation_psi, outdir)
print('Images Generated Successfully')

In [ ]:
!python dataset_tool.py --source=./results/generated --dest=./results/gan_images.zip